In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.applications import ResNet50  # Using ResNet50

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define the directory paths
base_dir = '/content/XRAY-Dataset_v2/XPAge01_RGB/XP'
images_dir = os.path.join(base_dir, 'JPGs')

# Load CSV file paths
train_csv = os.path.join(base_dir, 'trainingdata.csv')
test_csv = os.path.join(base_dir, 'testdata.csv')

# Load CSV data
train_data = pd.read_csv(train_csv)
test_data = pd.read_csv(test_csv)

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=images_dir,
    x_col='filenames',
    y_col='age',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=images_dir,
    x_col='filenames',
    y_col='age',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

# Build a ResNet50 model for regression (age prediction)
base_model = ResNet50(
    include_top=False,
    input_shape=(224, 224, 3),
    weights='imagenet'
)

# Add custom layers on top for regression
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  # Pooling layer
x = layers.Dense(224, activation='relu')(x)  # Fully connected layer
x = layers.Dense(1)(x)  # Output layer for continuous value (age prediction)

model = models.Model(inputs=base_model.input, outputs=x)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Compile the model for regression
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Define learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))  # Ensure the learning rate is a float

lr_scheduler = LearningRateScheduler(scheduler)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate the model
test_loss, test_mae = model.evaluate(test_generator)
print(f"Age Estimation Mean Absolute Error: {test_mae:.2f} years")
print(f"Age Estimation Mean Absolute Loss: {test_loss:.2f}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/XRAY-Dataset_v2/XPAge01_RGB/XP/trainingdata.csv'

In [ ]:
import zipfile
import os

# Define the name of the uploaded file
zip_file_name = 'XPAge01_RGB.zip'  # Replace with the name of your zip file

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/XRAY-Dataset_v2')  # Directory to extract to

In [ ]:
!rm -rf '/content/XPAge01_RGB.zip'